In [1]:
# I/O
import os
os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "1" # export NUMEXPR_NUM_THREADS=1

# Dataset
import numpy as np
import netCDF4 as nc

# Plot
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
matplotlib.use('Agg')

# Time
import time

# Video
from IPython.display import Video

In [2]:
def filename_constructor(time_index, case, var):
    return str(f"/data/cloud2023/vvmData{case}/archive{var}-{str(time_index).zfill(6)}.nc")

casenames = ["/rce_walker_15k_05m_p3",
             "/rce_walker_15k_1m_p3",
             "/rce_walker_1k_1m_p3",
             "/rce_walker_1k_2m_p3"]
varnames = ["/exp.C.LandSurface",
            "/exp.C.Surface",
            "/exp.L.Dynamic",
            "/exp.L.Radiation",
            "/exp.L.Thermodynamic"]
os.getcwd()

'/data/cloud2023/r12229008/HW4'

In [3]:
class GIFs:
    def __init__(self, case_idx):
        self.fig = plt.figure(figsize=(16, 9), dpi = 160)
        self.axes_3 = [self.fig.add_axes([0.10, 0.05, 0.32, 0.40]),
                       self.fig.add_axes([0.43, 0.05, 0.02, 0.40])]
        self.axes_4 = [self.fig.add_axes([0.54, 0.05, 0.32, 0.40]),
                       self.fig.add_axes([0.87, 0.05, 0.02, 0.40])]
        self.axes_1 = [self.fig.add_axes([0.10, 0.50, 0.32, 0.40]),
                       self.fig.add_axes([0.43, 0.50, 0.02, 0.40])]
        self.axes_2 = [self.fig.add_axes([0.54, 0.50, 0.32, 0.40]),
                       self.fig.add_axes([0.87, 0.50, 0.02, 0.40])]
        self.axes_2[0].sharex(self.axes_1[0])
        self.axes_3[0].sharex(self.axes_1[0])
        self.axes_4[0].sharex(self.axes_1[0])
        self.axes_2[0].sharey(self.axes_1[0])
        self.axes_3[0].sharey(self.axes_1[0])
        self.axes_4[0].sharey(self.axes_1[0])
        # self.ax1 = self.fig.add_axes([0.1,0.330,
        #                               0.800,0.550])
        # self.ax2 = self.fig.add_axes([0.1,0.080,
        #                               0.800,0.250])
        self.ID = case_idx
        
    def run(self):
        
        def init():
            dataset = nc.Dataset(filename_constructor(str(0).zfill(6), casenames[self.ID], varnames[-1]))
            self.xx, self.zz = np.meshgrid(dataset.variables["xc"][:]/1e3, dataset.variables["zc"][:11]/1e3)
            th = np.mean(dataset.variables["th"][-1, :11, :, :], axis = 1)
            qv = np.mean(dataset.variables["qv"][-1, :11, :, :], axis = 1)
            ql = np.mean(dataset.variables["qc"][-1, :11, :, :], axis = 1) + np.mean(dataset.variables["qr"][-1, :11, :, :], axis = 1)
            thv = th * (1 + 0.608 * qv - ql)
            
          
            # Fig.1
            self.plot1 = self.axes_1[0].contourf(self.xx, self.zz, thv - np.mean(thv, axis = 1).reshape(-1,1),
                                                 levels = np.linspace(-1,1,21),
                                                 extend = "both",
                                                 cmap = "bwr")
            plt.colorbar(self.plot1, cax = self.axes_1[1])
            self.axes_1[0].set_title(r"$\theta_v-\bar{\theta}_v(z)$")
            self.axes_1[0].set_xticks(np.linspace(0,1024,5), ["0", "256", "512", "768", "1024"])
            self.axes_1[0].set_xlim(0,1024)
            self.axes_1[0].set_yticks(np.linspace(0,2,5), ["0", "0.5", "1", "1.5", "2"])
            self.axes_1[0].set_ylim(0,2)
            # Fig.2
            self.plot2 = self.axes_2[0].contourf(self.xx, self.zz, th - np.mean(th, axis = 1).reshape(-1,1),
                                                 levels = np.linspace(-1,1,21),
                                                 extend = "both",
                                                 cmap = "bwr")
            plt.colorbar(self.plot2, cax = self.axes_2[1])
            self.axes_2[0].set_title(r"$\theta-\bar{\theta}(z)$")
            # Fig.3
            self.plot3 = self.axes_3[0].contourf(self.xx, self.zz, qv,
                                                 levels = np.linspace(0.012,0.020,21),
                                                 extend = "both",
                                                 cmap = "Reds")
            plt.colorbar(self.plot3, cax = self.axes_3[1])
            self.axes_3[0].set_title(r"$q_v$")
            # Fig.4
            self.plot4 = self.axes_4[0].contourf(self.xx, self.zz, ql,
                                                 levels = 51,
                                                 extend = "both",
                                                 cmap = "Reds")
            self.axes_4[0].set_title(r"$q_l$")
            plt.colorbar(self.plot4, cax = self.axes_4[1])
            # for idx, ax in enumerate(self.axes.flatten()):
            #     match idx:
            #         case 0:
            #             im = ax.contourf(self.xx, self.zz, thv, 
            #                              levels = np.linspace(295, 305, 51), 
            #                              extend = "both", 
            #                              cmap = "bwr")
            #             cbar = plt.colorbar(im)
            #             ax.set_title("$\\theta_v$")
            #             pass
            #         case 1:
            #             im = ax.contourf(self.xx, self.zz, th, 
            #                              levels = np.linspace(295, 305, 51), 
            #                              extend = "both", 
            #                              cmap = "bwr")
            #             cbar = plt.colorbar(im)
            #             ax.set_title("$\\theta$")
            #             pass
            #         case 2:
            #             im = ax.contourf(self.xx, self.zz, qv, 
            #                              levels = 32, 
            #                              extend = "both", 
            #                              cmap = "bwr")
            #             cbar = plt.colorbar(im)
            #             ax.set_title("$q_v$")
            #             pass
            #         case 3:
            #             im = ax.contourf(self.xx, self.zz, ql, 
            #                              levels = 32, 
            #                              extend = "both", 
            #                              cmap = "bwr")
            #             cbar = plt.colorbar(im)
            #             ax.set_title("$q_l$")
            #             pass
            return 0
        def update(time):
            try:
                self.plot1.remove()
                self.plot2.remove()
                self.plot3.remove()
                self.plot4.remove()
            except:
                pass
            dataset = nc.Dataset(filename_constructor(str(time).zfill(6), casenames[self.ID], varnames[-1]))
            th = np.mean(dataset.variables["th"][-1, :11, :, :], axis = 1)
            qv = np.mean(dataset.variables["qv"][-1, :11, :, :], axis = 1)
            ql = np.mean(dataset.variables["qc"][-1, :11, :, :], axis = 1) + np.mean(dataset.variables["qr"][-1, :11, :, :], axis = 1)
            thv = th * (1 + 0.608 * qv - ql)
            
            
            self.plot1 = self.axes_1[0].contourf(self.xx, self.zz, thv - np.mean(thv, axis = 1).reshape(-1,1),
                                                 levels = np.linspace(-1,1,21),
                                                 extend = "both",
                                                 cmap = "bwr")
            self.plot2 = self.axes_2[0].contourf(self.xx, self.zz, th - np.mean(th, axis = 1).reshape(-1,1),
                                                 levels = np.linspace(-1,1,21),
                                                 extend = "both",
                                                 cmap = "bwr")
            self.plot3 = self.axes_3[0].contourf(self.xx, self.zz, qv,
                                                 levels = np.linspace(0.012,0.020,21),
                                                 extend = "both",
                                                 cmap = "Reds")
            self.plot4 = self.axes_4[0].contourf(self.xx, self.zz, ql,
                                                 levels = 51,
                                                 extend = "both",
                                                 cmap = "Reds")
            self.fig.suptitle(f'Iterations : {time}')
            # for idx, ax in enumerate(self.axes.flatten()):
            #     match idx:
            #         case 0:
            #             pass
            #         case 1:
            #             pass
            #         case 2:
            #             pass
            #         case 3:
            #             pass
            return 0
        self.anim = animation.FuncAnimation(fig = self.fig, 
                                            func = update,
                                            frames = 1200, 
                                            init_func = init, 
                                            interval = 30,
                                            blit = False)
        self.anim.save("test.mp4")
        return None

In [4]:
test = GIFs(1)
test.run()


In [5]:
Video("test.mp4")